# 1. 네이버 영화 리뷰 데이터에 대한 이해와 전처리

In [ ]:
!pip install konlpy

In [ ]:
import torch

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
import pandas as pd
import urllib.request
import matplotlib.pyplot as plt
import re
from konlpy.tag import Okt
from tqdm import tqdm
import numpy as np

In [ ]:
# !wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt
# !wget https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="ratings_train.txt")
# urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="ratings_test.txt")

### 데이터 확인하기

- pd.read_table() 함수를 이용하여, ``ratings_train.txt``, ``ratings_test.txt`` 파일을 읽어와주세요. 변수명은 *train_data*, *test_data* 로 해주세요.

- 생성된 pandas data frame의 처음 10개의 행을 출력해주세요.

- 행의 길이를 출력해주세요

In [ ]:
## TODO ##

## 전처리 단계

중복 제거, null값, 공백 제거, 한글과 공백 이외의 문자 제거 등을 수행합니다.

In [ ]:
train_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
train_data['document'] = train_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
train_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
train_data = train_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(train_data))

In [ ]:
test_data.drop_duplicates(subset = ['document'], inplace=True) # document 열에서 중복인 내용이 있다면 중복 제거
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","") # 정규 표현식 수행
test_data['document'] = test_data['document'].str.replace('^ +', "") # 공백은 empty 값으로 변경
test_data['document'].replace('', np.nan, inplace=True) # 공백은 Null 값으로 변경
test_data = test_data.dropna(how='any') # Null 값 제거
print('전처리 후 테스트용 샘플의 개수 :',len(test_data))

## 토큰화

In [ ]:
import pickle

토큰화 작업은 시간이 꽤 걸립니다. 그래서 피클 파일들을 준비해 두었어요. 

In [ ]:
START_FROM_SCRATCH = False

stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']

if START_FROM_SCRATCH:
    # 10분쯤 걸려요..!
    okt = Okt()
    X_train = []
    for sentence in tqdm(train_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_train.append(stopwords_removed_sentence)
    X_test = []
    for sentence in tqdm(test_data['document']):
        tokenized_sentence = okt.morphs(sentence, stem=True) # 토큰화
        stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
        X_test.append(stopwords_removed_sentence)
    
    # X_train, X_test 객체 저장하기
    with open("X_train.pkl", "wb") as f:
        pickle.dump(X_train, f)
    with open("X_test.pkl", "wb") as f:
        pickle.dump(X_test, f)

# X_train, X_test 객체 불러오기
with open("X_train.pkl", "rb") as f:
    X_train = pickle.load(f)
with open("X_test.pkl", "rb") as f:
    X_test = pickle.load(f)

### Word to index 생성하기

In [ ]:
from collections import defaultdict

In [ ]:
word_count = defaultdict(int)

for tokens in tqdm(X_train):
    for token in tokens:
        word_count[token] += 1

word_count = sorted(word_count.items(), key=lambda x: x[1], reverse=True)

모든 단어를 다 쓰지 말고, 빈도가 가장 높은 20000개의 단어만 써봅시다. 

In [ ]:
vocab_size = 20000
## TODO ##

UNK, PAD 토큰을 추가해 봅시다.

In [ ]:
pad = 0
unk = 1

pad_token = "[PAD]"
unk_token = "[UNK]"

In [ ]:
word_count.insert(pad, (pad_token,-1))
word_count.insert(unk, (unk_token,-1))
vocab_size += 2
print(list(word_count))

w2i 딕셔너리를 정의해봅시다.

In [ ]:
w2i = {} 
for pair in tqdm(word_count):
    if pair[0] not in w2i:
        w2i[pair[0]] = len(w2i)
print(w2i)

In [ ]:
keys, values = list(w2i.keys()), list(w2i.values())
keys[0], values[0]

## 데이터 준비하기

데이터로더를 쓰기 전, 연습 먼저 해봅시다.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

쉬운 데이터 처리를 위해, 문장의 길이를 앞에서부터 30 tokens로 제한합시다.

string으로 되어있는 데이터를 w2i dictionary를 활용하여 정수로 바꾸어 줍시다. 

w2i dictionary에 없다면, 어떻게 해야 할까요?


In [ ]:
max_length = 30

In [ ]:
X_train_int = []
for sentence in tqdm(X_train):
    ## TODO ##

In [ ]:
X_train_int[0]

torch의 pad_sequence를 활용하여, padding 을 해 봅시다. 

``batch_first`` 와 ``padding_value`` argument들을 활용해보세요. 

In [ ]:
from torch.nn.utils.rnn import pad_sequence

In [ ]:
## TODO ##
X_train_pad = None

In [ ]:
X_train_pad[0], train_data["document"][0]

라벨(y) 도 같이 준비합시다.

In [ ]:
y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

In [ ]:
len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
class MovieReviewDataset(Dataset):
    def __init__(self, X, y, max_length = max_length, pad = pad):
        self.X = self.preprocess_data(X, max_length, pad = pad)
        self.y = torch.Tensor(y)
        
    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    def preprocess_data(self, X, max_length = max_length, pad = pad):
        data = []

        ## TODO ##
        #1. 최대 길이보다 길면 자르기
        #2. padding
        #3. 필요하다면, 텐서 타입으로 변경
        
        return data

In [ ]:
datasets = {}
datasets["train"] = MovieReviewDataset(X_train, y_train)
datasets["test"] = MovieReviewDataset(X_test, y_test)

In [ ]:
datasets["train"][0]

In [ ]:
dataloaders = {}
dataloaders["train"]= DataLoader(datasets["train"], batch_size=64, shuffle=True)
dataloaders["test"]= DataLoader(datasets["test"], batch_size=64, shuffle=True)

In [ ]:
next(iter(dataloaders["train"]))

# 2. LSTM으로 네이버 영화 리뷰 감성 분류하기

모델을 구성하고, device에 올려봅니다. 

In [ ]:
import torch.nn as nn

In [ ]:
class MovieReviewModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size, num_layers = 1):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(input_size = embedding_dim, hidden_size = hidden_size, num_layers = num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        embeddings = self.embedding(x)
        output, (hidden, cell) = self.rnn(embeddings)
        final_output = torch.sigmoid(self.linear(output[:, -1, :]))
        return final_output

In [ ]:
model = MovieReviewModel(vocab_size = vocab_size, embedding_dim = 100, hidden_size = 128, num_layers = 1)
model.to(device)

In [ ]:
sample_x, sample_y = next(iter(dataloaders["train"]))

In [ ]:
sample_x.shape, sample_y.shape

In [ ]:
len(dataloaders["train"])

모델을 training 시켜봅시다.

In [ ]:
learning_rate=1e-3
epochs = 10
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)
criterion = nn.BCELoss()

model.train()
for e in range(1, epochs+1):
    train_loss = 0.0
    for x,y in dataloaders["train"]:

        # TODO #

        # 1. forward

        # 2. backward
        
    epoch_loss = None
    print(f"EPOCH {e}: {epoch_loss}")

test 데이터셋에서 정확도를 산출해봅시다.

In [ ]:
with torch.no_grad():
  test_loss = 0.0
  test_correct = 0
  test_total = 0
  for x,y in dataloaders["test"]:

    ## TODO ##

    # 1. 모델 아웃풋 산출
    
    # 2. 맞은 개수, 총 개수 업데이트

  print(f"accuracy: {test_correct/test_total}")

# 3. 리뷰 예측해보기

In [ ]:
def sentiment_predict(new_sentence):
  print(new_sentence)

  # 전처리 과정
  okt = Okt()
  new_sentence = new_sentence.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
  new_sentence = new_sentence.replace('^ +', "")
  new_sentence = okt.morphs(new_sentence, stem=True) # 토큰화
  new_sentence = [word for word in new_sentence if not word in stopwords] # 불용어 제거
  new_sentence = new_sentence[:max_length]
  new_sentence = list(map(lambda w:w2i[w] if w in w2i else w2i['[UNK]'], new_sentence))
  new_sentence = torch.LongTensor(new_sentence).to(device).unsqueeze(0)
  
  print(f"tokenized: {new_sentence}")

  score = float(model(new_sentence))
  if(score > 0.5):
    print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
  else:
    print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [ ]:
sentiment_predict('이 영화 개꿀잼 ㅋㅋㅋ')

In [ ]:
sentiment_predict('이 영화 핵노잼 ㅠㅠ')

In [ ]:
sentiment_predict('이딴게 영화냐 ㅉㅉ')

In [ ]:
sentiment_predict('감독 뭐하는 놈이냐?')

In [ ]:
sentiment_predict('와 개쩐다 정말 세계관 최강자들의 영화다')